In [1]:
%matplotlib inline

In [2]:
cd ..

C:\Projects\python\recommender


In [3]:
from functools import partial
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [5]:
from datasets import SeqStackoverflow
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss, simple_weight_loss, trans_weight_loss

In [6]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
NEG_SAMPLE = 5
item_path = Path("./inputs/stackoverflow/item.csv")

In [7]:
db = SeqStackoverflow(data_path=item_path, min_user=4)
db

Original user size: 75572
Filtered user size: 14795
Original item dataframe shape: (299845, 9)
Filtered item dataframe shape: (217223, 9)
Remove number of questions less than 10 date
Filter item dateframe shape: (217146, 9)


In [8]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE,
             neg_sample=NEG_SAMPLE)

## Create Criterion

In [9]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [10]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x0000011390E70AE8>, 1, 1)

In [11]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x0000011390F0ED08>, 1, 1, 1)

In [12]:
simple_weight_loss_callback = partial(simple_weight_loss, LINEAR_REG, EMB_REG)
simple_weight_loss_callback

functools.partial(<function simple_weight_loss at 0x0000011390F0ED90>, 1, 1)

In [13]:
trans_weight_loss_callback = partial(trans_weight_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_weight_loss_callback

functools.partial(<function trans_weight_loss at 0x0000011390F0EE18>, 1, 1, 1)

## Train model

### Hyper-parameter

In [14]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

## Train fm model

In [15]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [22]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [23]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [24]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [18]:
fm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [19]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [25]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_weight_loss_callback)

In [20]:
fm_learner.fit(epoch=3,
               log_dir=get_log_dir(ds_type='simple_stackoverflow',
                                   model_type='fm'))

  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 165667.38797941408
Epoch 0 step 1: training accuarcy: 0.3044
Epoch 0 step 1: training loss: 160522.81291822562
Epoch 0 step 2: training accuarcy: 0.33480000000000004
Epoch 0 step 2: training loss: 155331.54568408176
Epoch 0 step 3: training accuarcy: 0.3816
Epoch 0 step 3: training loss: 150645.502428096
Epoch 0 step 4: training accuarcy: 0.4128
Epoch 0 step 4: training loss: 145525.92265924456
Epoch 0 step 5: training accuarcy: 0.4803
Epoch 0 step 5: training loss: 141141.7625060359
Epoch 0 step 6: training accuarcy: 0.5015000000000001
Epoch 0 step 6: training loss: 136641.41357274924
Epoch 0 step 7: training accuarcy: 0.5422
Epoch 0 step 7: training loss: 132610.45741318355
Epoch 0 step 8: training accuarcy: 0.5813
Epoch 0 step 8: training loss: 128733.55214231642
Epoch 0 step 9: training accuarcy: 0.5937
Epoch 0 step 9: training loss: 124958.7235743972
Epoch 0 step 10: training accuarcy: 0.6092000000000001
Epoch 0 step 10: training loss: 12080

Epoch 0 step 84: training accuarcy: 0.9331
Epoch 0 step 84: training loss: 20351.050122196255
Epoch 0 step 85: training accuarcy: 0.9395
Epoch 0 step 85: training loss: 20128.068385486138
Epoch 0 step 86: training accuarcy: 0.9282
Epoch 0 step 86: training loss: 19584.23711541897
Epoch 0 step 87: training accuarcy: 0.9442
Epoch 0 step 87: training loss: 19477.78651007922
Epoch 0 step 88: training accuarcy: 0.9269000000000001
Epoch 0 step 88: training loss: 19082.57262595649
Epoch 0 step 89: training accuarcy: 0.9348000000000001
Epoch 0 step 89: training loss: 18675.749836577408
Epoch 0 step 90: training accuarcy: 0.9415
Epoch 0 step 90: training loss: 18406.56658929334
Epoch 0 step 91: training accuarcy: 0.9414
Epoch 0 step 91: training loss: 18059.431024871345
Epoch 0 step 92: training accuarcy: 0.9436
Epoch 0 step 92: training loss: 17740.08509029554
Epoch 0 step 93: training accuarcy: 0.9454
Epoch 0 step 93: training loss: 17399.009047873413
Epoch 0 step 94: training accuarcy: 0.932

 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [05:58<11:56, 358.40s/it]

Epoch: 1
Epoch 1 step 94: training loss: 17253.000294836093
Epoch 1 step 95: training accuarcy: 0.9386
Epoch 1 step 95: training loss: 16929.44694080417
Epoch 1 step 96: training accuarcy: 0.9470000000000001
Epoch 1 step 96: training loss: 16618.178810533795
Epoch 1 step 97: training accuarcy: 0.9502
Epoch 1 step 97: training loss: 16417.661590538908
Epoch 1 step 98: training accuarcy: 0.9473
Epoch 1 step 98: training loss: 16143.834062044281
Epoch 1 step 99: training accuarcy: 0.9475
Epoch 1 step 99: training loss: 15896.39660381337
Epoch 1 step 100: training accuarcy: 0.9490000000000001
Epoch 1 step 100: training loss: 15748.38926558424
Epoch 1 step 101: training accuarcy: 0.9419000000000001
Epoch 1 step 101: training loss: 15386.377592670276
Epoch 1 step 102: training accuarcy: 0.9540000000000001
Epoch 1 step 102: training loss: 15376.393116589657
Epoch 1 step 103: training accuarcy: 0.9312
Epoch 1 step 103: training loss: 15011.044883610624
Epoch 1 step 104: training accuarcy: 0.94

Epoch 1 step 176: training accuarcy: 0.9887
Epoch 1 step 176: training loss: 5971.446969047685
Epoch 1 step 177: training accuarcy: 0.9888
Epoch 1 step 177: training loss: 5884.635030377054
Epoch 1 step 178: training accuarcy: 0.9916
Epoch 1 step 178: training loss: 5825.836682667761
Epoch 1 step 179: training accuarcy: 0.9918
Epoch 1 step 179: training loss: 5793.321270356484
Epoch 1 step 180: training accuarcy: 0.9883000000000001
Epoch 1 step 180: training loss: 5686.94758819889
Epoch 1 step 181: training accuarcy: 0.9914000000000001
Epoch 1 step 181: training loss: 5687.011143785813
Epoch 1 step 182: training accuarcy: 0.9893000000000001
Epoch 1 step 182: training loss: 5563.323907078093
Epoch 1 step 183: training accuarcy: 0.9919
Epoch 1 step 183: training loss: 5534.222165220891
Epoch 1 step 184: training accuarcy: 0.9889
Epoch 1 step 184: training loss: 5424.203185828366
Epoch 1 step 185: training accuarcy: 0.9937
Epoch 1 step 185: training loss: 5367.806292228768
Epoch 1 step 18

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [11:51<05:56, 356.91s/it]

Epoch: 2
Epoch 2 step 188: training loss: 5207.265124418872
Epoch 2 step 189: training accuarcy: 0.9912000000000001
Epoch 2 step 189: training loss: 5137.740812057923
Epoch 2 step 190: training accuarcy: 0.9938
Epoch 2 step 190: training loss: 5085.0555033591545
Epoch 2 step 191: training accuarcy: 0.9931000000000001
Epoch 2 step 191: training loss: 5022.475002910604
Epoch 2 step 192: training accuarcy: 0.9939
Epoch 2 step 192: training loss: 4962.457580573859
Epoch 2 step 193: training accuarcy: 0.9942000000000001
Epoch 2 step 193: training loss: 4918.621780876371
Epoch 2 step 194: training accuarcy: 0.993
Epoch 2 step 194: training loss: 4849.670847300937
Epoch 2 step 195: training accuarcy: 0.9955
Epoch 2 step 195: training loss: 4823.15943835334
Epoch 2 step 196: training accuarcy: 0.9933000000000001
Epoch 2 step 196: training loss: 4763.021790635826
Epoch 2 step 197: training accuarcy: 0.9929
Epoch 2 step 197: training loss: 4689.585151847931
Epoch 2 step 198: training accuarcy: 0

Epoch 2 step 270: training accuarcy: 1.0
Epoch 2 step 270: training loss: 2138.124518308894
Epoch 2 step 271: training accuarcy: 0.9994000000000001
Epoch 2 step 271: training loss: 2103.30946783866
Epoch 2 step 272: training accuarcy: 1.0
Epoch 2 step 272: training loss: 2084.806539043963
Epoch 2 step 273: training accuarcy: 0.999
Epoch 2 step 273: training loss: 2061.948415090914
Epoch 2 step 274: training accuarcy: 0.9999
Epoch 2 step 274: training loss: 2055.1590531309125
Epoch 2 step 275: training accuarcy: 0.9986
Epoch 2 step 275: training loss: 2019.6285273970925
Epoch 2 step 276: training accuarcy: 0.9998
Epoch 2 step 276: training loss: 2013.8092851145
Epoch 2 step 277: training accuarcy: 0.9988
Epoch 2 step 277: training loss: 1984.727633646739
Epoch 2 step 278: training accuarcy: 0.9997
Epoch 2 step 278: training loss: 1961.2753536597208
Epoch 2 step 279: training accuarcy: 1.0
Epoch 2 step 279: training loss: 1949.05788211908
Epoch 2 step 280: training accuarcy: 0.9995
Epoch

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [17:47<00:00, 356.55s/it]


In [26]:
fm_learner.fit(epoch=3,
               log_dir=get_log_dir(ds_type='weight_stackoverflow',
                                   model_type='fm'))

  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 201267.01323947922
Epoch 0 step 1: training accuarcy: 0.2868
Epoch 0 step 1: training loss: 192904.08661832023
Epoch 0 step 2: training accuarcy: 0.34790000000000004
Epoch 0 step 2: training loss: 185050.9434924109
Epoch 0 step 3: training accuarcy: 0.39890000000000003
Epoch 0 step 3: training loss: 177390.06584664635
Epoch 0 step 4: training accuarcy: 0.4494
Epoch 0 step 4: training loss: 170233.2167723767
Epoch 0 step 5: training accuarcy: 0.5005000000000001
Epoch 0 step 5: training loss: 161337.98932542282
Epoch 0 step 6: training accuarcy: 0.5667
Epoch 0 step 6: training loss: 154429.88840470806
Epoch 0 step 7: training accuarcy: 0.6111
Epoch 0 step 7: training loss: 147550.57053682234
Epoch 0 step 8: training accuarcy: 0.6662
Epoch 0 step 8: training loss: 142155.28672222613
Epoch 0 step 9: training accuarcy: 0.6998000000000001
Epoch 0 step 9: training loss: 136540.6399461447
Epoch 0 step 10: training accuarcy: 0.7302000000000001
Epoch 0 ste

Epoch 0 step 84: training accuarcy: 0.9985
Epoch 0 step 84: training loss: 23808.829168096105
Epoch 0 step 85: training accuarcy: 0.9976
Epoch 0 step 85: training loss: 23483.941547617942
Epoch 0 step 86: training accuarcy: 0.998
Epoch 0 step 86: training loss: 23133.286202027226
Epoch 0 step 87: training accuarcy: 0.999
Epoch 0 step 87: training loss: 22892.989787752485
Epoch 0 step 88: training accuarcy: 0.9982000000000001
Epoch 0 step 88: training loss: 22608.51072493714
Epoch 0 step 89: training accuarcy: 0.999
Epoch 0 step 89: training loss: 22395.188046523275
Epoch 0 step 90: training accuarcy: 0.9981000000000001
Epoch 0 step 90: training loss: 22135.751630016235
Epoch 0 step 91: training accuarcy: 0.9983000000000001
Epoch 0 step 91: training loss: 21876.646549715144
Epoch 0 step 92: training accuarcy: 0.9986
Epoch 0 step 92: training loss: 21670.650033173933
Epoch 0 step 93: training accuarcy: 0.9985
Epoch 0 step 93: training loss: 21404.523897124775
Epoch 0 step 94: training ac

 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [05:59<11:59, 359.68s/it]

Epoch: 1
Epoch 1 step 94: training loss: 21177.200945431392
Epoch 1 step 95: training accuarcy: 0.9995
Epoch 1 step 95: training loss: 20979.134364941914
Epoch 1 step 96: training accuarcy: 0.9996
Epoch 1 step 96: training loss: 20804.850597347526
Epoch 1 step 97: training accuarcy: 0.9989
Epoch 1 step 97: training loss: 20594.936166317028
Epoch 1 step 98: training accuarcy: 0.9988
Epoch 1 step 98: training loss: 20376.69277913169
Epoch 1 step 99: training accuarcy: 0.9997
Epoch 1 step 99: training loss: 20161.61100274686
Epoch 1 step 100: training accuarcy: 0.9991000000000001
Epoch 1 step 100: training loss: 20025.544135266195
Epoch 1 step 101: training accuarcy: 0.9985
Epoch 1 step 101: training loss: 19865.46342637483
Epoch 1 step 102: training accuarcy: 0.998
Epoch 1 step 102: training loss: 19723.00470830216
Epoch 1 step 103: training accuarcy: 0.9981000000000001
Epoch 1 step 103: training loss: 19523.435445045267
Epoch 1 step 104: training accuarcy: 0.9982000000000001
Epoch 1 ste

Epoch 1 step 178: training loss: 12802.40172813432
Epoch 1 step 179: training accuarcy: 1.0
Epoch 1 step 179: training loss: 12745.240163237848
Epoch 1 step 180: training accuarcy: 1.0
Epoch 1 step 180: training loss: 12695.210112532448
Epoch 1 step 181: training accuarcy: 0.9999
Epoch 1 step 181: training loss: 12698.079433262585
Epoch 1 step 182: training accuarcy: 0.9985
Epoch 1 step 182: training loss: 12600.41668554977
Epoch 1 step 183: training accuarcy: 0.9996
Epoch 1 step 183: training loss: 12565.675429981968
Epoch 1 step 184: training accuarcy: 0.9995
Epoch 1 step 184: training loss: 12477.439825444853
Epoch 1 step 185: training accuarcy: 1.0
Epoch 1 step 185: training loss: 12402.94862374809
Epoch 1 step 186: training accuarcy: 1.0
Epoch 1 step 186: training loss: 12382.622871603588
Epoch 1 step 187: training accuarcy: 1.0
Epoch 1 step 187: training loss: 12308.77604860306
Epoch 1 step 188: training accuarcy: 1.0
Epoch 1: train loss 15732.679137420417, train accuarcy 0.99903

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [12:09<06:02, 362.71s/it]

Epoch: 2
Epoch 2 step 188: training loss: 12300.236456950415
Epoch 2 step 189: training accuarcy: 0.9993000000000001
Epoch 2 step 189: training loss: 12200.950215147088
Epoch 2 step 190: training accuarcy: 1.0
Epoch 2 step 190: training loss: 12167.176559371544
Epoch 2 step 191: training accuarcy: 0.9995
Epoch 2 step 191: training loss: 12101.416119692034
Epoch 2 step 192: training accuarcy: 1.0
Epoch 2 step 192: training loss: 12039.16645057354
Epoch 2 step 193: training accuarcy: 1.0
Epoch 2 step 193: training loss: 11989.235659360593
Epoch 2 step 194: training accuarcy: 1.0
Epoch 2 step 194: training loss: 11927.153164028949
Epoch 2 step 195: training accuarcy: 1.0
Epoch 2 step 195: training loss: 11874.240504642654
Epoch 2 step 196: training accuarcy: 1.0
Epoch 2 step 196: training loss: 11824.391694250335
Epoch 2 step 197: training accuarcy: 1.0
Epoch 2 step 197: training loss: 11796.360362880008
Epoch 2 step 198: training accuarcy: 0.9995
Epoch 2 step 198: training loss: 11720.97

Epoch 2 step 275: training loss: 8369.18828946939
Epoch 2 step 276: training accuarcy: 0.9999
Epoch 2 step 276: training loss: 8321.273058349925
Epoch 2 step 277: training accuarcy: 1.0
Epoch 2 step 277: training loss: 8298.559005412475
Epoch 2 step 278: training accuarcy: 1.0
Epoch 2 step 278: training loss: 8269.125641629324
Epoch 2 step 279: training accuarcy: 0.9995
Epoch 2 step 279: training loss: 8211.382642890887
Epoch 2 step 280: training accuarcy: 1.0
Epoch 2 step 280: training loss: 8176.705935688254
Epoch 2 step 281: training accuarcy: 1.0
Epoch 2 step 281: training loss: 8141.26506977212
Epoch 2 step 282: training accuarcy: 1.0
Epoch 2: train loss 10073.563062301644, train accuarcy 0.999770998954773
Epoch 2: valid loss 8143.10337157626, valid accuarcy 0.9994051456451416


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [18:09<00:00, 361.97s/it]


In [27]:
del fm_model
T.cuda.empty_cache()

### Train HRM FM Model

In [28]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [29]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [30]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [23]:
hrm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [31]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_loss_callback)

In [ ]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_weight_loss_callback)

In [32]:
hrm_learner.fit(epoch=3,
                log_dir=get_log_dir('simple_stackoverflow', 'hrm'))

  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 178408.48944542778
Epoch 0 step 1: training accuarcy: 0.5234
Epoch 0 step 1: training loss: 173944.06400449394
Epoch 0 step 2: training accuarcy: 0.5077
Epoch 0 step 2: training loss: 169842.53513716318
Epoch 0 step 3: training accuarcy: 0.49820000000000003
Epoch 0 step 3: training loss: 164192.18646111747
Epoch 0 step 4: training accuarcy: 0.5177
Epoch 0 step 4: training loss: 159937.99615528205
Epoch 0 step 5: training accuarcy: 0.5089
Epoch 0 step 5: training loss: 155624.50188310235
Epoch 0 step 6: training accuarcy: 0.5074000000000001
Epoch 0 step 6: training loss: 151740.96887853605
Epoch 0 step 7: training accuarcy: 0.4958
Epoch 0 step 7: training loss: 146556.3743770176
Epoch 0 step 8: training accuarcy: 0.5113
Epoch 0 step 8: training loss: 143461.46983378765
Epoch 0 step 9: training accuarcy: 0.4988
Epoch 0 step 9: training loss: 139739.39706399117
Epoch 0 step 10: training accuarcy: 0.4884
Epoch 0 step 10: training loss: 134877.2318309

Epoch 0 step 85: training accuarcy: 0.4959
Epoch 0 step 85: training loss: 20731.94942008032
Epoch 0 step 86: training accuarcy: 0.5295
Epoch 0 step 86: training loss: 20715.781427207236
Epoch 0 step 87: training accuarcy: 0.49470000000000003
Epoch 0 step 87: training loss: 20130.07280277498
Epoch 0 step 88: training accuarcy: 0.5139
Epoch 0 step 88: training loss: 19818.639053560124
Epoch 0 step 89: training accuarcy: 0.5099
Epoch 0 step 89: training loss: 19335.44697160584
Epoch 0 step 90: training accuarcy: 0.5159
Epoch 0 step 90: training loss: 19043.855683456924
Epoch 0 step 91: training accuarcy: 0.5165000000000001
Epoch 0 step 91: training loss: 18680.43631977283
Epoch 0 step 92: training accuarcy: 0.524
Epoch 0 step 92: training loss: 18351.616376642396
Epoch 0 step 93: training accuarcy: 0.5206000000000001
Epoch 0 step 93: training loss: 16345.99030138254
Epoch 0 step 94: training accuarcy: 0.5207450224791266
Epoch 0: train loss 65671.42420970614, train accuarcy 0.504197955131

 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [06:11<12:23, 371.92s/it]

Epoch: 1
Epoch 1 step 94: training loss: 17293.625049746122
Epoch 1 step 95: training accuarcy: 0.5603
Epoch 1 step 95: training loss: 16950.176599689454
Epoch 1 step 96: training accuarcy: 0.5644
Epoch 1 step 96: training loss: 16723.639007434376
Epoch 1 step 97: training accuarcy: 0.5651
Epoch 1 step 97: training loss: 16445.692095079554
Epoch 1 step 98: training accuarcy: 0.5661
Epoch 1 step 98: training loss: 16367.643600016207
Epoch 1 step 99: training accuarcy: 0.5424
Epoch 1 step 99: training loss: 16017.413141002744
Epoch 1 step 100: training accuarcy: 0.5542
Epoch 1 step 100: training loss: 15736.604455415858
Epoch 1 step 101: training accuarcy: 0.5636
Epoch 1 step 101: training loss: 15387.01878076182
Epoch 1 step 102: training accuarcy: 0.5724
Epoch 1 step 102: training loss: 15301.909772094428
Epoch 1 step 103: training accuarcy: 0.5547000000000001
Epoch 1 step 103: training loss: 14940.400976389348
Epoch 1 step 104: training accuarcy: 0.5719000000000001
Epoch 1 step 104: t

KeyboardInterrupt: 

In [ ]:
hrm_learner.fit(epoch=3,
                log_dir=get_log_dir('weight_stackoverflow', 'hrm'))

In [29]:
del hrm_model
T.cuda.empty_cache()

### Train PRME FM Model

In [30]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [31]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [32]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [33]:
prme_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [34]:
prme_learner.fit(epoch=5,log_dir=get_log_dir('seq_stackoverflow', 'prme'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 163625.31334404234
Epoch 0 step 1: training accuarcy: 0.493
Epoch 0 step 1: training loss: 159053.84386484543
Epoch 0 step 2: training accuarcy: 0.496
Epoch 0 step 2: training loss: 154421.37391361973
Epoch 0 step 3: training accuarcy: 0.51
Epoch 0 step 3: training loss: 150078.63620001235
Epoch 0 step 4: training accuarcy: 0.5135
Epoch 0 step 4: training loss: 144752.72417866022
Epoch 0 step 5: training accuarcy: 0.5345
Epoch 0 step 5: training loss: 141069.2796469489
Epoch 0 step 6: training accuarcy: 0.513
Epoch 0 step 6: training loss: 137534.93500742718
Epoch 0 step 7: training accuarcy: 0.498
Epoch 0 step 7: training loss: 132837.28214766507
Epoch 0 step 8: training accuarcy: 0.5165
Epoch 0 step 8: training loss: 129455.17493600384
Epoch 0 step 9: training accuarcy: 0.5005000000000001
Epoch 0 step 9: training loss: 125055.68127879125
Epoch 0 step 10: training accuarcy: 0.51
Epoch 0 step 10: training loss: 120937.32056976783
Epoch 0 step 11:

 20%|██████████████████████████████████▌                                                                                                                                          | 1/5 [01:22<05:31, 82.92s/it]

Epoch: 1
Epoch 1 step 94: training loss: 8490.246713243258
Epoch 1 step 95: training accuarcy: 0.529
Epoch 1 step 95: training loss: 8182.121943406677
Epoch 1 step 96: training accuarcy: 0.538
Epoch 1 step 96: training loss: 7983.988104257445
Epoch 1 step 97: training accuarcy: 0.531
Epoch 1 step 97: training loss: 7751.978461277806
Epoch 1 step 98: training accuarcy: 0.5295
Epoch 1 step 98: training loss: 7552.896497317663
Epoch 1 step 99: training accuarcy: 0.5305
Epoch 1 step 99: training loss: 7368.974533635669
Epoch 1 step 100: training accuarcy: 0.513
Epoch 1 step 100: training loss: 7083.361314707041
Epoch 1 step 101: training accuarcy: 0.5255
Epoch 1 step 101: training loss: 6914.690266067468
Epoch 1 step 102: training accuarcy: 0.523
Epoch 1 step 102: training loss: 6784.328299345862
Epoch 1 step 103: training accuarcy: 0.513
Epoch 1 step 103: training loss: 6534.229203209648
Epoch 1 step 104: training accuarcy: 0.5435
Epoch 1 step 104: training loss: 6336.477428611949
Epoch 1

 40%|█████████████████████████████████████████████████████████████████████▏                                                                                                       | 2/5 [02:49<04:11, 83.94s/it]

...

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                     | 3/5 [04:12<02:47, 83.66s/it]

Epoch: 3
Epoch 3 step 282: training loss: 1409.6966998501161
Epoch 3 step 283: training accuarcy: 0.587
Epoch 3 step 283: training loss: 1411.8633999444946
Epoch 3 step 284: training accuarcy: 0.5865
Epoch 3 step 284: training loss: 1381.4901869120238
Epoch 3 step 285: training accuarcy: 0.612
Epoch 3 step 285: training loss: 1388.13925670535
Epoch 3 step 286: training accuarcy: 0.5985
Epoch 3 step 286: training loss: 1382.698158967659
Epoch 3 step 287: training accuarcy: 0.602
Epoch 3 step 287: training loss: 1389.9223615245196
Epoch 3 step 288: training accuarcy: 0.603
Epoch 3 step 288: training loss: 1383.2993944610123
Epoch 3 step 289: training accuarcy: 0.6075
Epoch 3 step 289: training loss: 1398.890204059772
Epoch 3 step 290: training accuarcy: 0.5875
Epoch 3 step 290: training loss: 1390.7510149107393
Epoch 3 step 291: training accuarcy: 0.595
Epoch 3 step 291: training loss: 1387.5828021902194
Epoch 3 step 292: training accuarcy: 0.59
Epoch 3 step 292: training loss: 1398.1483

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                  | 4/5 [05:36<01:23, 83.80s/it]

Epoch: 4
Epoch 4 step 376: training loss: 1360.9900585763228
Epoch 4 step 377: training accuarcy: 0.5785
Epoch 4 step 377: training loss: 1359.267329781073
Epoch 4 step 378: training accuarcy: 0.6015
Epoch 4 step 378: training loss: 1361.7449540081498
Epoch 4 step 379: training accuarcy: 0.595
Epoch 4 step 379: training loss: 1345.396363767901
Epoch 4 step 380: training accuarcy: 0.6075
Epoch 4 step 380: training loss: 1346.4767924012322
Epoch 4 step 381: training accuarcy: 0.594
Epoch 4 step 381: training loss: 1332.4465988798675
Epoch 4 step 382: training accuarcy: 0.599
Epoch 4 step 382: training loss: 1341.1260411701455
Epoch 4 step 383: training accuarcy: 0.604
Epoch 4 step 383: training loss: 1356.9727228753327
Epoch 4 step 384: training accuarcy: 0.593
Epoch 4 step 384: training loss: 1338.5238653857832
Epoch 4 step 385: training accuarcy: 0.6055
Epoch 4 step 385: training loss: 1356.2163716840184
Epoch 4 step 386: training accuarcy: 0.5885
Epoch 4 step 386: training loss: 1354.

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [07:02<00:00, 84.56s/it]


In [41]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [13]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [14]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [15]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [16]:
trans_learner.compile(train_col='base',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=simple_loss_callback)

In [17]:
trans_learner.fit(epoch=5, log_dir=get_log_dir('seq_stackoverflow', 'trans'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 164958.5357854265
Epoch 0 step 1: training accuarcy: 0.518
Epoch 0 step 1: training loss: 160368.91942496574
Epoch 0 step 2: training accuarcy: 0.5035000000000001
Epoch 0 step 2: training loss: 155344.1250532723
Epoch 0 step 3: training accuarcy: 0.5175
Epoch 0 step 3: training loss: 151314.12961604865
Epoch 0 step 4: training accuarcy: 0.513
Epoch 0 step 4: training loss: 146592.98273384754
Epoch 0 step 5: training accuarcy: 0.496
Epoch 0 step 5: training loss: 142728.94892648255
Epoch 0 step 6: training accuarcy: 0.4975
Epoch 0 step 6: training loss: 138132.09020402856
Epoch 0 step 7: training accuarcy: 0.518
Epoch 0 step 7: training loss: 133968.60335181077
Epoch 0 step 8: training accuarcy: 0.5075000000000001
Epoch 0 step 8: training loss: 130519.33231767353
Epoch 0 step 9: training accuarcy: 0.5075000000000001
Epoch 0 step 9: training loss: 127024.94322712412
Epoch 0 step 10: training accuarcy: 0.4865
Epoch 0 step 10: training loss: 122531.0

 20%|██████████████████████████████████▌                                                                                                                                          | 1/5 [01:31<06:04, 91.09s/it]

Epoch: 1
Epoch 1 step 94: training loss: 8468.06361467602
Epoch 1 step 95: training accuarcy: 0.619
Epoch 1 step 95: training loss: 8237.7399516149
Epoch 1 step 96: training accuarcy: 0.622
Epoch 1 step 96: training loss: 8025.693425424888
Epoch 1 step 97: training accuarcy: 0.615
Epoch 1 step 97: training loss: 7775.535208786701
Epoch 1 step 98: training accuarcy: 0.629
Epoch 1 step 98: training loss: 7581.423166052558
Epoch 1 step 99: training accuarcy: 0.618
Epoch 1 step 99: training loss: 7346.008004475699
Epoch 1 step 100: training accuarcy: 0.631
Epoch 1 step 100: training loss: 7157.674886435413
Epoch 1 step 101: training accuarcy: 0.614
Epoch 1 step 101: training loss: 6975.372647907814
Epoch 1 step 102: training accuarcy: 0.64
Epoch 1 step 102: training loss: 6807.763966372768
Epoch 1 step 103: training accuarcy: 0.61
Epoch 1 step 103: training loss: 6616.774350835358
Epoch 1 step 104: training accuarcy: 0.629
Epoch 1 step 104: training loss: 6428.667614136943
Epoch 1 step 105

 40%|█████████████████████████████████████████████████████████████████████▏                                                                                                       | 2/5 [03:02<04:33, 91.18s/it]

...

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                     | 3/5 [04:33<03:02, 91.09s/it]

Epoch: 3
Epoch 3 step 282: training loss: 1321.837596538467
Epoch 3 step 283: training accuarcy: 0.7255
Epoch 3 step 283: training loss: 1325.6472751078177
Epoch 3 step 284: training accuarcy: 0.74
Epoch 3 step 284: training loss: 1327.626364497246
Epoch 3 step 285: training accuarcy: 0.733
Epoch 3 step 285: training loss: 1365.6448296152348
Epoch 3 step 286: training accuarcy: 0.7295
Epoch 3 step 286: training loss: 1354.4467639057405
Epoch 3 step 287: training accuarcy: 0.726
Epoch 3 step 287: training loss: 1329.9979958535655
Epoch 3 step 288: training accuarcy: 0.7375
Epoch 3 step 288: training loss: 1302.4285655950382
Epoch 3 step 289: training accuarcy: 0.7335
Epoch 3 step 289: training loss: 1352.3322442710369
Epoch 3 step 290: training accuarcy: 0.725
Epoch 3 step 290: training loss: 1366.7640848759622
Epoch 3 step 291: training accuarcy: 0.7205
Epoch 3 step 291: training loss: 1316.3711810879238
Epoch 3 step 292: training accuarcy: 0.7365
Epoch 3 step 292: training loss: 1341.

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                  | 4/5 [06:06<01:31, 91.69s/it]

Epoch: 4
Epoch 4 step 376: training loss: 1193.7661577366157
Epoch 4 step 377: training accuarcy: 0.7565000000000001
Epoch 4 step 377: training loss: 1213.703070783885
Epoch 4 step 378: training accuarcy: 0.763
Epoch 4 step 378: training loss: 1246.8831791309726
Epoch 4 step 379: training accuarcy: 0.7415
Epoch 4 step 379: training loss: 1194.7704351348823
Epoch 4 step 380: training accuarcy: 0.753
Epoch 4 step 380: training loss: 1239.4857555369022
Epoch 4 step 381: training accuarcy: 0.7515000000000001
Epoch 4 step 381: training loss: 1221.842278153663
Epoch 4 step 382: training accuarcy: 0.7495
Epoch 4 step 382: training loss: 1194.8138116524128
Epoch 4 step 383: training accuarcy: 0.759
Epoch 4 step 383: training loss: 1240.3861053817477
Epoch 4 step 384: training accuarcy: 0.7465
Epoch 4 step 384: training loss: 1234.7102872370465
Epoch 4 step 385: training accuarcy: 0.7515000000000001
Epoch 4 step 385: training loss: 1246.9405335407016
Epoch 4 step 386: training accuarcy: 0.755
E

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [07:39<00:00, 92.05s/it]


In [ ]:
del trans_model
T.cuda.empty_cache()